In [ ]:
import tensorflow as tf
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [ ]:
# Parameters
learning_rate = 0.001
training_iters = 500
batch_size = 128
display_step = 10

# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

In [ ]:
def W_initializer():
    return tf.initializers.orthogonal()

def b_initializer():
    return tf.initializers.zeros()

def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation    
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x) 


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],padding= "SAME")


# Create model
def conv_net(x, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    with tf.variable_scope("conv1") as scope:
        W = tf.get_variable("W",
                            shape = [5,5,1,32],
                            initializer= W_initializer())
        
        b = tf.get_variable("b",
                            shape = [32],
                            initializer = b_initializer())
        
        conv1 = conv2d(x,W,b)
        # Max Pooling (down-sampling)
        conv1 = maxpool2d(conv1, k=2)
        

    # Convolution Layer
    with tf.variable_scope("conv2") as scope:
        W = tf.get_variable("W",
                            shape = [5,5,32,64],
                            initializer= W_initializer())
        
        b = tf.get_variable("b",
                            shape = [64],
                            initializer = b_initializer())
        
        conv2 = conv2d(conv1,W,b)
        # Max Pooling (down-sampling)
        conv2 = maxpool2d(conv2, k=2)
        

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    with tf.variable_scope("FC1") as scope:  
        W = tf.get_variable("W",
                            shape = [7*7*64, 1024],
                            initializer= W_initializer())
        
        b = tf.get_variable("b",
                            shape = [1024],
                            initializer = b_initializer())
        
        fc1 = tf.layers.flatten(conv2)
        fc1 = tf.add(tf.matmul(fc1,W),b)
        fc1 = tf.nn.relu(fc1)
        
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)
    
    # Output, class prediction
    with tf.variable_scope("Softmax") as scope:
         W = tf.get_variable("W",
                            shape = [1024, n_classes],
                            initializer= W_initializer())
        
         b = tf.get_variable("b",
                            shape = [n_classes],
                            initializer = b_initializer())
        
         out = tf.add(tf.matmul(fc1,W),b)
        
    return out

In [ ]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

# Construct model
logits = conv_net(x, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [ ]:
# Start training
saver = tf.train.Saver()

with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_iters+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={x: batch_x,
                                                                 y: batch_y,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    save_path = saver.save(sess,"/home/jacob/Learning_resources-/codes/python_tutorial/checkpoints/")

In [ ]:
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, "/home/jacob/model.ckpt")
    loss, acc = sess.run([loss_op, accuracy], feed_dict={x:  mnist.test.images[:256],
                                                                   y: mnist.test.labels[:256],
                                                                 keep_prob: 1.0})
    print("Step " + str(step) + ", Minibatch Loss= " + \
          "{:.4f}".format(loss) + ", Training Accuracy= " + \
          "{:.3f}".format(acc))
